In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm
from keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler

In [2]:
gpus = tf.config.list_physical_devices(device_type = 'GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [3]:
with open('./pickle/pickle_corr/대형주_20221206_1년_10개_06.pkl', 'rb') as f:
    col_list = pickle.load(f) # 상관 계수에 따른 컬럼 리스트

In [4]:
modelpath="./model/대형주_20221206_1년_lstm_30ep_64bs_5pa_06cor.hdf5"
model = load_model(modelpath)

In [5]:
def stock_dataframe(code):
    with open(f'./pickle/pickle_df/update/대형주_20221207_10개_update_{code}.pkl', 'rb') as f:
        pred_df = pickle.load(f)
    return pred_df

def predict_stock(pred_df, col_list):
    c_list = list(col_list.index)
    X_pred_df = pred_df.drop(['pct_label','날짜'],axis=1)
    X_pred_df = X_pred_df[c_list]
    # min_abs_scaler = MinMaxScaler()
    min_abs_scaler = MaxAbsScaler()
    X_pred_sc = min_abs_scaler.fit_transform(X_pred_df)
    X_pred = X_pred_sc.reshape(X_pred_sc.shape[0], model.input.shape[1], 1)

    predict = model.predict(X_pred)
    
    return predict

In [6]:
pred_stock_df = stock_dataframe('005930')

In [41]:
s= pd.DataFrame(pred_stock_df.loc[0:1])
s

,날짜,시간,시가,고가,저가,종가,거래량,거래대금,누적체결매수수량,누적체결매도수량,...,헝가리BUX내역_저가,헝가리BUX내역_거래량,헝가리BUX내역_변동,날짜,호주SPASX내역_종가,호주SPASX내역_오픈,호주SPASX내역_고가,호주SPASX내역_저가,호주SPASX내역_거래량,호주SPASX내역_변동
0,20221207,901,58800,59300,58800,59300,592414,34877550000,566511,24951,...,44207.42,0.0,1.49,20221207.0,7229.4,7291.3,7291.3,7220.6,810630000.0,-0.85
1,20221207,902,59200,59500,59100,59100,332547,19723750000,742998,181011,...,44207.42,0.0,1.49,20221207.0,7229.4,7291.3,7291.3,7220.6,810630000.0,-0.85


In [42]:
predict_ = predict_stock(s, col_list) # 종목 예측

1/1 [==============================] - 2s 2s/step


In [53]:
predict = []

In [54]:
predict.append(predict_[0])
predict.append(predict_[1])

In [59]:
predict[0][0]

0.7214829

In [60]:
cost = 10000000 # 잔고
stock_list = ['005930','373220','207940','000660','051910','247540','091990','066970','293490','028300']

for code in stock_list: # 한 종목 거래 진행

    pred_stock_df = stock_dataframe(code) # 종목 데이터 프레임
    predict_ = predict_stock(pred_stock_df, col_list) # 종목 예측

    print('거래 진행 종목 코드 : ' + str(code))
    stock_cost = cost / len(stock_list) # 종목별 매수 금액
    standard_cost = cost / len(stock_list)
    num = 0 # 가지고 있는 주
    profit = 0
    end = 1000000000000000000000
    sell = 0
    for x in range(len(predict_)):
        
        end_cost = pred_stock_df.loc[x,'종가']   # 종가
        high_cost = pred_stock_df.loc[x,'고가']   # 고가
        
        # num = 매수금액 / 종가
        buy_num = stock_cost // end_cost

        if (predict_[x][0] < predict_[x][1]) & (end_cost < high_cost) & (buy_num > 0):
            if sell == 0:   
                print('+++++++++++++++++++++++++++++++ 매수 위치 +++++++++++++++++++++++++++++++')
                print('종목별 매수 금액 : ' + str(stock_cost) + ' 종가 : ' + str(end_cost) + ' 고가 : ' + str(high_cost) + ' 매수 수량 : ' + str(buy_num))
                print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
                stock_cost = stock_cost - (end_cost*buy_num) # 매수
                num += buy_num
                end = end_cost
            elif sell > 0:
                if (sell*0.98) > end_cost:
                    print('+++++++++++++++++++++++++++++++ 매수 위치 +++++++++++++++++++++++++++++++')
                    print('종목별 매수 금액 : ' + str(stock_cost) + ' 종가 : ' + str(end_cost) + ' 고가 : ' + str(high_cost) + ' 매수 수량 : ' + str(buy_num))
                    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
                    stock_cost = stock_cost - (end_cost*buy_num) # 매수
                    num += buy_num
                    end = end_cost
                    sell = 0
                else:
                    continue
            
        ### 구매시 종가보다 몇 퍼센트 이상 증가했으면 바로 팔아라
        elif ((end*1.03) < end_cost) & (num > 0):
            print('------------------------------- 매도 위치 -------------------------------')
            print('종목별 매수 금액 : ' + str(stock_cost) + ' 종가 : ' + str(end_cost) + ' 고가 : ' + str(high_cost) + ' 매도 수량 : ' + str(num))
            print('------------------------------------------------------------------------')
            stock_cost = stock_cost + (end_cost * num) # 종가로 매도 
            tax = (end_cost * num)*0.005
            num -= num
            profit = stock_cost - standard_cost - tax
            print('매수 & 매도 후 금액 : ' + str(stock_cost))
            print('세금 확인 : ' + str(tax))
            print('수익금 확인 : ' + str(profit))
            stock_cost = cost / len(stock_list)
            print('------------------------------------------------------------------------')

        
        elif (predict_[x][0] > predict_[x][1]) & (num > 0):
            print('------------------------------- 매도 위치 -------------------------------')
            print('종목별 매수 금액 : ' + str(stock_cost) + ' 종가 : ' + str(end_cost) + ' 고가 : ' + str(high_cost) + ' 매도 수량 : ' + str(num))
            print('------------------------------------------------------------------------')
            stock_cost = stock_cost + (end_cost * num) # 종가로 매도 
            tax = (end_cost * num)*0.005
            num -= num
            profit = stock_cost - standard_cost - tax
            print('매수 & 매도 후 금액 : ' + str(stock_cost))
            print('세금 확인 : ' + str(tax))
            print('수익금 확인 : ' + str(profit))
            stock_cost = cost / len(stock_list)
            print('------------------------------------------------------------------------')

        elif (num > 0) & (x == (len(predict_)-2)):
            
            print('**************************** 장 마감 전 매도 **************************')
            print('종목별 매수 금액 : ' + str(stock_cost) + ' 종가 : ' + str(end_cost) + ' 고가 : ' + str(high_cost) + ' 매도 수량 : ' + str(num))
            print('**********************************************************************')
            stock_cost = stock_cost + (end_cost * num) # 종가로 매도 
            tax = (end_cost * num)*0.005
            num -= num 
            profit = stock_cost - standard_cost - tax
            print('매수 & 매도 후 금액 : ' + str(stock_cost))
            print('세금 확인 : ' + str(tax))
            print('수익금 확인 : ' + str(profit))
            stock_cost = cost / len(stock_list)
            print('**********************************************************************')
        
        else:
            print('종목별 매수 금액 : ' + str(stock_cost) + ' 종가 : ' + str(end_cost) + ' 고가 : ' + str(high_cost) + ' 보유 수량 : ' + str(num))
            
cost = cost + profit
print('거래 후 최종 잔고 : ' + str(cost))
#  잔고가 얼마냐? + 수익  - 손실

12/12 [==============================] - 0s 13ms/step
거래 진행 종목 코드 : 005930
종목별 매수 금액 : 1000000.0 종가 : 59300.0 고가 : 59300.0 보유 수량 : 0
+++++++++++++++++++++++++++++++ 매수 위치 +++++++++++++++++++++++++++++++
종목별 매수 금액 : 1000000.0 종가 : 59100.0 고가 : 59500.0 매수 수량 : 16.0
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
종목별 매수 금액 : 54400.0 종가 : 59100.0 고가 : 59200.0 보유 수량 : 16.0
종목별 매수 금액 : 54400.0 종가 : 59100.0 고가 : 59200.0 보유 수량 : 16.0
종목별 매수 금액 : 54400.0 종가 : 59400.0 고가 : 59600.0 보유 수량 : 16.0
종목별 매수 금액 : 54400.0 종가 : 59300.0 고가 : 59400.0 보유 수량 : 16.0
종목별 매수 금액 : 54400.0 종가 : 59100.0 고가 : 59300.0 보유 수량 : 16.0
종목별 매수 금액 : 54400.0 종가 : 59200.0 고가 : 59200.0 보유 수량 : 16.0
종목별 매수 금액 : 54400.0 종가 : 59200.0 고가 : 59300.0 보유 수량 : 16.0
종목별 매수 금액 : 54400.0 종가 : 59200.0 고가 : 59300.0 보유 수량 : 16.0
종목별 매수 금액 : 54400.0 종가 : 59200.0 고가 : 59300.0 보유 수량 : 16.0
종목별 매수 금액 : 54400.0 종가 : 59300.0 고가 : 59300.0 보유 수량 : 16.0
종목별 매수 금액 : 54400.0 종가 : 59300.0 고가 : 59300.0 보유 수량 : 16.0
종목별 매수 금액 : 54

In [ ]:
cost

10008966.75